In [1]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import operator as op
import pandas as pd
import warnings
import pickle
from copy import deepcopy
from matplotlib import pyplot as plt
from cmapPy.pandasGEXpress.parse import parse

import ivbase
import partners_projects_2

from invivoplatform.data_parser.molecular_dataframe import MolecularDataFrame
from ivbase.utils.commons import is_smiles_iterable

os.chdir(partners_projects_2.__path__[0] + '/..')

RDKit WARNING: [15:39:07] Enabling RDKit 2019.09.2 jupyter extensions


In [2]:
BASE_PATH = 'data/clue/'
DATA_PATH = BASE_PATH + 'GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx'
DATA_INFO = BASE_PATH + 'GSE92742_Broad_LINCS_pert_info.txt'
COL_TO_KEEP = '_A375_6H'
NEW_FILE = COL_TO_KEEP + '.csv'
SMILES_COL = 'canonical_smiles'

In [3]:
data_info = pd.read_csv(DATA_INFO, sep='\t')
is_BRD = ['BRD-' in elem for elem in data_info['pert_id'].values]
data_info_mols = data_info[is_BRD]
data_info_mols = data_info_mols.set_index('pert_id')
data_info_mols

,pert_iname,pert_type,is_touchstone,inchi_key_prefix,inchi_key,canonical_smiles,pubchem_cid
pert_id,,,,,,,
BRD-A00100033,nifurtimox,trt_cp,1,ARFHIAQFJWUCFH,ARFHIAQFJWUCFH-UHFFFAOYSA-N,CC1CS(=O)(=O)CCN1N=Cc1ccc(o1)[N+]([O-])=O,6842999
BRD-A00150179,5-hydroxytryptophan,trt_cp,0,QSHLMQDRPXXYEE,QSHLMQDRPXXYEE-UHFFFAOYSA-N,NC(Cc1c[nH]c2cccc(O)c12)C(O)=O,589768
BRD-A00267231,hemado,trt_cp,1,KOCIMZNSNPOGOP,KOCIMZNSNPOGOP-UHFFFAOYSA-N,CCCCC#Cc1nc(NC)c2ncn(C3OC(CO)C(O)C3O)c2n1,4043357
BRD-A00420644,SA-3676,trt_cp,0,ASCBUEVCEVGOFP,ASCBUEVCEVGOFP-UHFFFAOYSA-N,CCN1C2C(C(=NC2Nc3ccccc13)OC)c4ccccc4,2853908
BRD-A00474148,BRD-A00474148,trt_cp,0,RCGAUPRLRFZAMS,RCGAUPRLRFZAMS-UHFFFAOYSA-N,Oc1ccc(cc1)N1CCN(CC1)[S+]([O-])(=O)c1ccc2NC(=O...,44825297
...,...,...,...,...,...,...,...
BRD-U88459701,atorvastatin,trt_cp,1,XUKUURHRXDUEBC,XUKUURHRXDUEBC-UHFFFAOYSA-N,CC(C)c1c(C(=O)Nc2ccccc2)c(c(-c2ccc(F)cc2)n1CCC...,-666
BRD-U88861852,YL-57,trt_cp,0,-666,-666,-666,-666
BRD-U88878891,BRD-U88878891,trt_cp,0,-666,-666,-666,-666


In [4]:
data = parse(DATA_PATH, ridx=[0])

In [5]:
cols = data.data_df.columns
unique_cols = deepcopy(np.unique([col.split(':')[0] for col in cols]))
del data
print(len(unique_cols))
for elem in unique_cols:
    print(elem)

1374
b'AML001_CD34_24H
b'AML001_CD34_6H
b'AML001_HL60_24H
b'AML001_HL60_6H
b'AML001_PC3_6H
b'AML001_U937_24H
b'AML001_U937_6H
b'ASG001_MCF7_24H
b'ASG001_MCF7_6H
b'ASG001_PC3_24H
b'ASG001_PC3_6H
b'BRAF001_A375_24H
b'BRAF001_A375_6H
b'BRAF001_HEK293T_24H
b'BRAF001_HEK293T_6H
b'CGS001_A375_96H
b'CGS001_A549_96H
b'CGS001_ASC_96H
b'CGS001_HA1E_96H
b'CGS001_HCC515_96H
b'CGS001_HEK293T_96H
b'CGS001_HEKTE_96H
b'CGS001_HEPG2_96H
b'CGS001_HT29_96H
b'CGS001_MCF7_144H
b'CGS001_MCF7_96H
b'CGS001_NCIH716_144H
b'CGS001_NPC_96H
b'CGS001_PC3_120H
b'CGS001_PC3_144H
b'CGS001_PC3_96H
b'CGS001_SHSY5Y_120H
b'CGS001_SKL_96H
b'CGS001_SW480_96H
b'CGS001_VCAP_120H
b'CGS001_VCAP_168H
b'CGS001_VCAP_24H
b'CGS001_VCAP_48H
b'CGS001_VCAP_6H
b'CGS001_VCAP_96H
b'CNS001_A375_24H
b'CNS001_A375_6H
b'CNS001_A375_96H
b'CNS001_A549_24H
b'CNS001_A549_6H
b'CNS001_A549_96H
b'CNS001_ASC_24H
b'CNS001_ASC_96H
b'CNS001_BT20_1H
b'CNS001_BT20_24H
b'CNS001_BT20_3H
b'CNS001_BT20_6H
b'CNS001_FIBRNPC_24H
b'CNS001_FIBRNPC_6H
b'CNS001_HA1E

b'KDD001_HA1E_96H
b'KDD001_HCC515_96H
b'KDD001_HEPG2_96H
b'KDD001_HT29_96H
b'KDD001_PC3_96H
b'KDD001_VCAP_120H
b'KDD002_A375_96H
b'KDD002_A549_96H
b'KDD002_HA1E_96H
b'KDD002_HCC515_96H
b'KDD002_HEPG2_96H
b'KDD002_HT29_96H
b'KDD002_MCF7_96H
b'KDD002_PC3_96H
b'KDD002_VCAP_120H
b'KDD003_A549_96H
b'KDD003_HA1E_96H
b'KDD003_HCC515_96H
b'KDD003_HEPG2_96H
b'KDD003_HT29_96H
b'KDD003_MCF7_96H
b'KDD003_PC3_96H
b'KDD003_VCAP_120H
b'KDD004_A375_96H
b'KDD004_A549_96H
b'KDD004_HA1E_96H
b'KDD004_HCC515_96H
b'KDD004_HEPG2_96H
b'KDD004_HT29_96H
b'KDD004_MCF7_96H
b'KDD004_PC3_96H
b'KDD004_VCAP_120H
b'KDD005_A375_96H
b'KDD005_A549_96H
b'KDD005_HA1E_96H
b'KDD005_HCC515_96H
b'KDD005_HT29_96H
b'KDD005_MCF7_96H
b'KDD005_PC3_96H
b'KDD005_VCAP_120H
b'KDD006_A375_96H
b'KDD006_A549_96H
b'KDD006_HA1E_96H
b'KDD006_HCC515_96H
b'KDD006_HEPG2_96H
b'KDD006_HT29_96H
b'KDD006_MCF7_96H
b'KDD006_PC3_96H
b'KDD006_VCAP_120H
b'KDD007_A375_96H
b'KDD007_A549_96H
b'KDD007_HA1E_96H
b'KDD007_HCC515_96H
b'KDD007_HT29_96H
b'KDD007_

In [6]:
cidx = [ii for ii, col in enumerate(cols) if (COL_TO_KEEP in col.split(':')[0]) and ('BRD' in col)]
print(len(cidx))

max_ridx = 12000
load_ridx = 1000
data2 = pd.DataFrame()
for start_ridx in np.arange(0, max_ridx, load_ridx):
    ridx = np.arange(start_ridx, start_ridx+load_ridx, 1)
    print(min(ridx), max(ridx))
    this_data = parse(DATA_PATH, ridx=ridx, cidx=cidx).data_df
    data2 = data2.append(this_data)

6403
0 999
1000 1999
2000 2999
3000 3999
4000 4999
5000 5999
6000 6999
7000 7999
8000 8999
9000 9999
10000 10999
11000 11999


In [7]:
data2.index

Index(['b'5720'', 'b'466'', 'b'6009'', 'b'2309'', 'b'387'', 'b'3553'',
       'b'427'', 'b'5898'', 'b'23365'', 'b'6657'',
       ...
       'b'27006'', 'b'57828'', 'b'56130'', 'b'58158'', 'b'56128'', 'b'10017'',
       'b'30819'', 'b'64111'', 'b'80329'', 'b'50834''],
      dtype='object', name='rid', length=12000)

In [8]:
new_idx = [idx if "b'" not in idx else idx[2:-1] for idx in data2.index]
new_cols = [col if "b'" not in col else col[2:-1] for col in data2.columns]
data2.index = new_idx
data2.columns = new_cols
data2

,CPC005_A375_6H:BRD-A85280935-003-01-7:10,CPC005_A375_6H:BRD-A07824748-001-02-6:10,CPC004_A375_6H:BRD-K20482099-001-01-1:10,CPC005_A375_6H:BRD-K62929068-001-03-3:10,CPC005_A375_6H:BRD-K43405658-001-01-8:10,CPC004_A375_6H:BRD-K03670461-001-02-0:10,CPC004_A375_6H:BRD-K36737713-001-01-6:10,CPC005_A375_6H:BRD-K51223576-001-01-3:10,CPC004_A375_6H:BRD-A14966924-001-03-0:10,CPC004_A375_6H:BRD-K79131256-001-08-8:10,...,BRAF001_A375_6H:BRD-K63675182-003-18-6:0.625,BRAF001_A375_6H:BRD-K63675182-003-18-6:0.15625,BRAF001_A375_6H:BRD-U73308409-000-01-9:10,BRAF001_A375_6H:BRD-U73308409-000-01-9:2.5,BRAF001_A375_6H:BRD-U73308409-000-01-9:0.625,BRAF001_A375_6H:BRD-U73308409-000-01-9:0.15625,BRAF001_A375_6H:BRD-K81418486-001-15-2:10,BRAF001_A375_6H:BRD-K81418486-001-15-2:2.5,BRAF001_A375_6H:BRD-K81418486-001-15-2:0.625,BRAF001_A375_6H:BRD-K81418486-001-15-2:0.15625
5720,0.773769,-0.645586,-5.449666,0.193408,1.006298,-5.388713,-1.000240,0.490110,0.063297,0.560929,...,-0.005215,0.174461,1.549974,1.204220,0.191044,1.241092,1.266400,1.218014,1.079661,0.889130
466,-0.818468,-0.810749,2.393775,-0.582243,0.455536,1.867731,-1.106092,0.595174,-0.962553,-0.656688,...,-0.027974,0.392942,-0.286208,-0.904040,-0.870842,-0.510538,-7.691465,-7.150860,-6.152345,-2.738755
6009,0.189572,0.459060,1.279790,-0.178977,0.631738,0.281383,-0.422545,-0.224163,0.521553,0.520286,...,0.033848,0.236814,-0.620936,-0.129555,0.792432,0.160836,-0.770763,-0.276317,0.090466,0.932443
2309,-0.146031,-0.224676,2.167868,-1.182025,-0.936414,1.378175,0.406279,-0.244783,0.182361,-0.315654,...,-0.234941,0.481425,1.705819,1.374210,1.702135,1.963918,-0.072071,0.765122,0.500579,-0.312492
387,-0.654002,-0.335681,2.333199,-1.012651,-1.213203,1.290522,-0.218671,-0.124029,0.572183,-0.187850,...,-0.445761,-0.514716,0.089809,0.238403,1.589008,-0.223807,0.945684,0.974351,0.470916,0.597323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10017,0.719733,1.132764,4.725786,0.601818,0.248587,1.376116,0.157545,-0.270624,-0.134903,-0.335120,...,0.052006,-0.069262,-0.079207,-0.135815,0.482430,0.195270,0.027923,-0.071364,-0.921938,-0.547217
30819,0.046575,2.250843,3.134251,1.452990,-0.071597,0.353774,0.386104,0.326849,-0.060763,-0.674900,...,0.014039,0.085872,0.438203,-0.160172,0.590631,0.539895,0.711804,0.208447,-0.148957,0.318163
64111,1.735312,3.361870,2.810047,-0.073817,0.623073,1.674171,-0.145533,-0.068347,0.257254,-0.665090,...,0.237947,0.568111,0.266715,0.300890,-0.299126,1.067907,-0.148996,-0.272246,-1.201684,-0.210973
80329,0.727611,2.054916,3.978644,0.318097,-0.127867,1.369819,-0.116350,0.064481,-0.195314,-0.533690,...,-0.203675,0.082351,0.853724,0.600848,-0.012614,0.559722,1.228856,1.111301,0.286133,-0.013337


In [9]:
df = data2.transpose()

mol_id, full_id, plate_id, target, cell_line, hours = [[] for ii in range(6)]

for idx in df.index:
    if 'BRD' in idx:
        start = idx.find('BRD-')
        end = start + 13
        if idx[start:end] in data_info_mols.index:
            mol_id.append(idx[start:end])
            full_id.append(idx)
            plate_id.append(idx.split(':')[0])
            plate_id_split = plate_id[-1].split('_')
            target.append(plate_id_split[0])
            cell_line.append(plate_id_split[1])
            hours.append(plate_id_split[2])
            
new_df = data_info_mols.loc[mol_id]
new_df['full_id'] = full_id
new_df['plate_id'] = plate_id
new_df['target'] = target
new_df['cell_line'] = cell_line
new_df['hours'] = hours

new_df

,pert_iname,pert_type,is_touchstone,inchi_key_prefix,inchi_key,canonical_smiles,pubchem_cid,full_id,plate_id,target,cell_line,hours
pert_id,,,,,,,,,,,,
BRD-A85280935,quinpirole,trt_cp,1,FTSUPYGMFAPCFZ,FTSUPYGMFAPCFZ-UHFFFAOYSA-N,CCCN1CCCC2Cc3n[nH]cc3CC21,-666,CPC005_A375_6H:BRD-A85280935-003-01-7:10,CPC005_A375_6H,CPC005,A375,6H
BRD-A07824748,flavanone,trt_cp,1,ZONYXWQDUYMKFB,ZONYXWQDUYMKFB-UHFFFAOYSA-N,O=C1CC(Oc2ccccc12)c3ccccc3,-666,CPC005_A375_6H:BRD-A07824748-001-02-6:10,CPC005_A375_6H,CPC005,A375,6H
BRD-K20482099,rutin,trt_cp,1,IKGXIBQEEMLURG,IKGXIBQEEMLURG-NVPNHPEKSA-N,C[C@@H]1O[C@@H](OC[C@H]2O[C@@H](Oc3c(oc4cc(O)c...,-666,CPC004_A375_6H:BRD-K20482099-001-01-1:10,CPC004_A375_6H,CPC004,A375,6H
BRD-K62929068,6-benzylaminopurine,trt_cp,1,NWBJYWHLCVSVIJ,NWBJYWHLCVSVIJ-UHFFFAOYSA-N,N(Cc1ccccc1)c1ncnc2nc[nH]c12,-666,CPC005_A375_6H:BRD-K62929068-001-03-3:10,CPC005_A375_6H,CPC005,A375,6H
BRD-K43405658,tyrphostin-AG-527,trt_cp,1,UMGQVUWXNOJOSJ,UMGQVUWXNOJOSJ-KMHUVPDISA-N,C[C@@H](NC(=O)/C(=C/c1ccc(O)c(O)c1)/C#N)c2ccccc2,-666,CPC005_A375_6H:BRD-K43405658-001-01-8:10,CPC005_A375_6H,CPC005,A375,6H
...,...,...,...,...,...,...,...,...,...,...,...,...
BRD-U73308409,vemurafenib,trt_cp,0,-666,-666,-666,-666,BRAF001_A375_6H:BRD-U73308409-000-01-9:0.15625,BRAF001_A375_6H,BRAF001,A375,6H
BRD-K81418486,vorinostat,trt_cp,1,WAEXFXRVDQXREF,WAEXFXRVDQXREF-UHFFFAOYSA-N,ONC(=O)CCCCCCC(=O)Nc1ccccc1,-666,BRAF001_A375_6H:BRD-K81418486-001-15-2:10,BRAF001_A375_6H,BRAF001,A375,6H
BRD-K81418486,vorinostat,trt_cp,1,WAEXFXRVDQXREF,WAEXFXRVDQXREF-UHFFFAOYSA-N,ONC(=O)CCCCCCC(=O)Nc1ccccc1,-666,BRAF001_A375_6H:BRD-K81418486-001-15-2:2.5,BRAF001_A375_6H,BRAF001,A375,6H


In [10]:
merged_df = new_df.reset_index(drop=True).join(df.reset_index(drop=True))
new_col_names = {col: 'value_' + str(col) for col in merged_df.columns if str(col).isdigit()}
merged_df = merged_df.rename(columns=new_col_names)
merged_df.columns

Index(['pert_iname', 'pert_type', 'is_touchstone', 'inchi_key_prefix',
       'inchi_key', 'canonical_smiles', 'pubchem_cid', 'full_id', 'plate_id',
       'target',
       ...
       'value_27006', 'value_57828', 'value_56130', 'value_58158',
       'value_56128', 'value_10017', 'value_30819', 'value_64111',
       'value_80329', 'value_50834'],
      dtype='object', length=12012)

In [11]:
smiles = merged_df[SMILES_COL].values.flatten()
is_smiles = is_smiles_iterable(smiles)
smiles_valid = smiles[is_smiles]
print(len(smiles_valid))
print(len(np.unique(smiles_valid)))

6361
5155


In [12]:
is_A375 = (merged_df['cell_line'].values == 'A375') & is_smiles
is_6H = (merged_df['hours'].values == '6H')
is_24H = (merged_df['hours'].values == '24H')
smiles_6H = smiles[is_A375 & is_6H]
print('6H : ', len(smiles_6H) - len(np.unique(smiles_6H)))
smiles_24H = smiles[is_A375 & is_24H]
print('24H : ', len(smiles_24H) - len(np.unique(smiles_24H)))

smiles_6H_24H = np.concatenate((smiles_6H, smiles_24H), axis=0)
print('6H-24H : ', len(smiles_6H_24H) - len(np.unique(smiles_6H_24H)))

6H :  1206
24H :  0
6H-24H :  1206


In [15]:
merged_df_no_duplicates = merged_df[is_smiles].drop_duplicates(subset=SMILES_COL)
merged_df_no_duplicates = merged_df_no_duplicates.rename(columns={SMILES_COL: 'SMILES'})
merged_df_no_duplicates.to_csv(BASE_PATH + NEW_FILE, index=False)

In [14]:
merged_df_no_duplicates.shape

(5155, 12012)